<a href="https://colab.research.google.com/github/grissharrisdennis/Machine-learning-Projects/blob/main/book_recommendation_using_k_nearest_neighbours.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-11-15 04:56:00--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  25.2MB/s    in 1.0s    

2023-11-15 04:56:02 (25.2 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
user_ratings = df_ratings.groupby('user', as_index = False)['rating'].count().rename(columns = { 'rating':'usercount'})
book_ratings = df_ratings.groupby('isbn', as_index = False)['rating'].count().rename(columns = {'rating':'titlecount'})
df_ratings = pd.merge(left = df_ratings, right = user_ratings, on = 'user')
df_ratings = pd.merge(left = df_ratings, right = book_ratings, on = 'isbn')

df = df_ratings.loc[(df_ratings['usercount'] >= 200) & (df_ratings['titlecount'] >= 100)]
df = df.drop(['usercount', 'titlecount'], axis = 1)
df = pd.merge(left = df, right = df_books, on = 'isbn')

In [ ]:
print(user_ratings,book_ratings)

          user  usercount
0            2          1
1            7          1
2            8         18
3            9          3
4           10          2
...        ...        ...
105278  278846          2
105279  278849          4
105280  278851         23
105281  278852          1
105282  278854          8

[105283 rows x 2 columns]                isbn  titlecount
0        0330299891           2
1        0375404120           2
2        0586045007           1
3        9022906116           2
4        9032803328           1
...             ...         ...
340551     cn113107           1
340552   ooo7156103           1
340553   §423350229           1
340554  ´3499128624           1
340555    Ô½crosoft           1

[340556 rows x 2 columns]


In [ ]:
df = df.drop_duplicates(['user', 'title'])

In [ ]:
df_pivot = pd.pivot(df, values = 'rating', index = 'title', columns = 'user' )

In [ ]:
df_pivot = df_pivot.fillna(0)

In [ ]:

df_matrix = csr_matrix(df_pivot.values)

In [ ]:
model = NearestNeighbors(algorithm = 'brute', metric = 'cosine')
model.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  x=df_pivot[df_pivot.index==book]
  if len(x) == 0:
        return ["Book not found in the dataset"]
  dist, ind = model.kneighbors(x, n_neighbors = 6)
  recommended_books = []
  reco_books = []
  dist = dist.flatten()
  ind = ind.flatten()
  for i in range(len(ind)):
      if i == 0:
        recommended_books.append(df_pivot.index[ind[i]])
      else:
        reco_book =df_pivot.index[ind[i]]
        reco_dist = dist[i]
        reco_books.append([reco_book, reco_dist])
  reco_books = reco_books[::-1]
  recommended_books.append(reco_books)
  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
